In [10]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import visdom

import mxnet as mx
from mxnet import gluon
from mxnet import autograd
from mxnet import image

import sys
sys.path.append('../../resuneta/src')
sys.path.append('../../resuneta/nn/loss')
sys.path.append('../../resuneta/models')
sys.path.append('../../')
sys.path.append('../MXNet-ResUNeta/')

from bound_dist import get_distance, get_boundary
from loss import Tanimoto_wth_dual
from resunet_d6_causal_mtskcolor_ddist import *
from resunet_d7_causal_mtskcolor_ddist import *

from datasets import *

In [11]:
def dice_coef(x, y):
    if type(x).__module__ == 'numpy':
        intersection = np.logical_and(x, y)
        return 2. * np.sum(intersection) / (np.sum(x) + np.sum(y))
    else:
        intersection = mx.ndarray.op.broadcast_logical_and(x, y)
        return 2. * mx.nd.sum(intersection) / (mx.nd.sum(x) + mx.nd.sum(y))

In [12]:
def visdom_visualize_batch(vis, img, extent, boundary, distance,
                           extent_pred, boundary_pred, distance_pred,
                           hsv, hsv_pred, title="Train images"):

    img, extent, boundary, distance = img.asnumpy(), extent.asnumpy(), boundary.asnumpy(), distance.asnumpy()
    extent_pred, boundary_pred = extent_pred.asnumpy(), boundary_pred.asnumpy()
    distance_pred, hsv, hsv_pred = distance_pred.asnumpy(), hsv.asnumpy(), hsv_pred.asnumpy()

    # put everything in one window
    batch_size, nchannels, nrows, ncols = img.shape
    padding = 10
    items = [img, hsv, hsv_pred, extent, extent_pred, 
             boundary, boundary_pred, distance, distance_pred]
    result = np.zeros((3, len(items)*nrows + (len(items)-1)*padding, batch_size*ncols + (batch_size-1)*padding))

    for j, item in enumerate(items):

        if item.shape[1] == 1:
            item = np.tile(item, (1,3,1,1)) * 255.

        if j == 1 or j == 2: # convert HSV to RGB
            item = np.moveaxis(item, 1, -1) * 255.
            for i in range(batch_size):
                item[i] = cv2.cvtColor(item[i].astype(np.uint8), cv2.COLOR_HSV2RGB)
            item = np.moveaxis(item, -1, 1)
            
        for i in range(batch_size):
            result[:, j*(nrows+padding):(j+1)*nrows+j*padding, i*(ncols+padding):(i+1)*ncols+i*padding] = item[i]
    vis.images(result, nrow=1, win=title, opts={'title': title})


In [13]:
def train_model(train_dataloader, model, tanimoto_dual, trainer, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    
    # training set
    for batch_i, (img, extent, boundary, distance, hsv) in enumerate(
        tqdm(train_dataloader, desc='Training epoch {}'.format(epoch))):
        
        with autograd.record():

            img = img.as_in_context(mx.gpu(args['gpu']))
            extent = extent.as_in_context(mx.gpu(args['gpu']))
            boundary = boundary.as_in_context(mx.gpu(args['gpu']))
            distance = distance.as_in_context(mx.gpu(args['gpu']))
            hsv = hsv.as_in_context(mx.gpu(args['gpu']))
            
            logits, bound, dist, convc = model(img)
            
            # multi-task loss
            # TODO: wrap this in a custom loss function / class
            loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent))
            loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary))
            loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance))
            loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv))
            
            loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
            
        loss.backward()
        trainer.step(args['batch_size'])
        
        # update metrics based on every batch
        cumulative_loss += mx.nd.sum(loss).asscalar()
        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits[:,[0],:,:] - 0.5)
        accuracy.update(extent, extent_predicted_classes)
        # f1 score
        prediction = logits[:,0,:,:].reshape(-1)
        probabilities = mx.nd.stack(1 - prediction, prediction, axis=1)
        f1.update(extent.reshape(-1), probabilities)
        # MCC metric
        mcc.update(extent.reshape(-1), probabilities)
        # Dice score
        dice.update(extent, extent_predicted_classes)
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc)

    return cumulative_loss, accuracy, f1, mcc, dice


In [14]:
def evaluate_model(val_dataloader, model, tanimoto_dual, epoch, args):
    
    # initialize metrics
    cumulative_loss = 0
    accuracy = mx.metric.Accuracy()
    f1 = mx.metric.F1()
    mcc = mx.metric.MCC()
    dice = mx.metric.CustomMetric(feval=dice_coef, name="Dice")
    
    # validation set
    for batch_i, (img, extent, boundary, distance, hsv) in enumerate(
        tqdm(val_dataloader, desc='Validation epoch {}'.format(epoch))):

        img = img.as_in_context(mx.gpu(args['gpu']))
        extent = extent.as_in_context(mx.gpu(args['gpu']))
        boundary = boundary.as_in_context(mx.gpu(args['gpu']))
        distance = distance.as_in_context(mx.gpu(args['gpu']))
        hsv = hsv.as_in_context(mx.gpu(args['gpu']))

        logits, bound, dist, convc = model(img)
            
        # multi-task loss
        # TODO: wrap this in a custom loss function / class
        loss_extent = mx.nd.sum(1 - tanimoto_dual(logits, extent))
        loss_boundary = mx.nd.sum(1 - tanimoto_dual(bound, boundary))
        loss_distance = mx.nd.sum(1 - tanimoto_dual(dist, distance))
        loss_hsv = mx.nd.sum(1 - tanimoto_dual(convc, hsv))

        loss = 0.25 * (loss_extent + loss_boundary + loss_distance + loss_hsv)
        
        # update metrics based on every batch
        cumulative_loss += mx.nd.sum(loss).asscalar()
        # accuracy
        extent_predicted_classes = mx.nd.ceil(logits[:,[0],:,:] - 0.5)
        accuracy.update(extent, extent_predicted_classes)
        # f1 score
        prediction = logits[:,0,:,:].reshape(-1)
        probabilities = mx.nd.stack(1 - prediction, prediction, axis=1)
        f1.update(extent.reshape(-1), probabilities)
        # MCC metric
        mcc.update(extent.reshape(-1), probabilities)
        # Dice score
        dice.update(extent, extent_predicted_classes)
        # TODO: eccentricity
        # TODO: ...
        
        if batch_i % args['visdom_every'] == 0:
            visdom_visualize_batch(args['visdom'], img, extent, boundary, distance,
                                   logits, bound, dist, hsv, convc, title="Val images")
        
    return cumulative_loss, accuracy, f1, mcc, dice

# New 10k samples in France

In [15]:
def run_france(splits_path='../data/splits/sherrie10k_planetImagery_splits.csv',
               epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
               n_classes=1, model_type='resunet-d6', month='april', 
               codes_to_keep=list(np.arange(1,29, dtype=np.uint8)), 
               image_suffix='.tif', gpu_id=0):
    
    
    # Set up names of directories and paths for saving
    if len(codes_to_keep) >= 24:
        classes = 'all'
    else:
        classes = 'subset'
    folder_name = model_type+'_'+month+'_class-'+classes+'_nfilter-'+str(n_filters)+'_bs-'+str(batch_size)+'_lr-'+str(lr)
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = os.path.join('../experiments/france/sherrie10k/', folder_name)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")
    
    # Directories for loading data
    month_dict = {'april': '2019_04', 'october': '2019_10', 'january': '2019_01'}
    month_folder = month_dict[month]
    image_directory = os.path.join(
        '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/', month_folder)
    label_directory = '../data/planet/france/sherrie10k/extent_labels/'

    # Visdom
    env_name = 'france_sherrie10k_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    # Arguments
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 20

    # Load train/val/test splits
    splits_df = pd.read_csv(splits_path)
    splits_df['image_id'] = splits_df['image_id'].astype(str).str.zfill(5)
    train_names = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names = splits_df[splits_df['fold'] == 'test']['image_id'].values

    train_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='train', image_names=train_names, classes=codes_to_keep,
        image_suffix=image_suffix)
    val_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='val', image_names=val_names, classes=codes_to_keep,
        image_suffix=image_suffix)
    test_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='test', image_names=test_names, classes=codes_to_keep,
        image_suffix=image_suffix)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [16]:
# # ============================ #
# # user-specified hyperparameters
# # ============================ #
# epochs = 100
# lr = 0.001
# lr_decay = None
# n_filters = 32
# batch_size = 8
# n_classes = 1
# model_type = 'resunet-d7'
# month = 'october'
# codes_to_keep = list(np.arange(1,28, dtype=np.uint8))
# gpu_id = 1
# # ============================ #

# run_france(splits_path='../data/splits/sherrie10k_planetImagery_splits_20x20.csv',
#            epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
#            n_classes=n_classes, model_type=model_type, month=month, image_suffix='_2019_10.tif',
#            codes_to_keep=codes_to_keep, gpu_id=gpu_id)

In [17]:
def run_france_all_images(splits_path='../data/splits/sherrie10k_planetImagery_splits.csv',
                          image_directory = '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/',
                          label_directory = '../data/planet/france/sherrie10k/extent_labels/',
                          epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
                          n_classes=1, model_type='resunet-d6', month='april', 
                          codes_to_keep=list(np.arange(1,29, dtype=np.uint8)), 
                          image_suffix='.tif', 
                          folder_suffix='',
                          boundary_kernel_size=3,
                          gpu_id=0):
    
    
    # Set up names of directories and paths for saving
    if len(codes_to_keep) >= 24:
        classes = 'all'
    elif codes_to_keep == list(range(1,10)) + [11,14,15,16,17,18,19,24,25,26,27]:
        classes = 'notreecrops'
    elif codes_to_keep == list(range(1,10)) + [11,14,15,16,17,18,19,21,24,25,26,27]:
        classes = 'notreeexceptvines'
    else:
        classes = 'subset'
    folder_name = model_type+'_'+month+'_class-'+classes+'_nfilter-'+str(n_filters)+ \
                  '_bs-'+str(batch_size)+'_lr-'+str(lr)+folder_suffix
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = os.path.join('../experiments/france/sherrie10k/', folder_name)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")
    
    # Visdom
    env_name = 'france_sherrie10k_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    # Arguments
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 20

    # Load train/val/test splits
    splits_df = pd.read_csv(splits_path)
    splits_df['image_id'] = splits_df['image_id'].astype(str).str.zfill(5)
    train_names_label = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names_label = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names_label = splits_df[splits_df['fold'] == 'test']['image_id'].values
    train_names, val_names, test_names = [], [], []
    for year_month in ['2019_10', '2019_07', '2019_04']:
        train_names = train_names + [year_month+'/'+x+'_'+year_month for x in train_names_label]
        val_names = val_names + [year_month+'/'+x+'_'+year_month for x in val_names_label]
        test_names = test_names + [year_month+'/'+x+'_'+year_month for x in test_names_label]

    train_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='train', image_names=train_names, 
        label_names=train_names_label, classes=codes_to_keep, image_suffix=image_suffix,
        boundary_kernel_size=boundary_kernel_size)
    val_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='val', image_names=val_names, 
        label_names=val_names_label, classes=codes_to_keep, image_suffix=image_suffix,
        boundary_kernel_size=boundary_kernel_size)
    test_dataset = PlanetDatasetWithClasses(
        image_directory, label_directory, fold='test', image_names=test_names, 
        label_names=test_names_label, classes=codes_to_keep, image_suffix=image_suffix,
        boundary_kernel_size=boundary_kernel_size)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [ ]:
# ============================ #
# user-specified hyperparameters
# ============================ #
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 16
batch_size = 8
n_classes = 1
model_type = 'resunet-d6'
month = 'aprJulOctSeparate'
codes_to_keep = list(range(1,10)) + [11,14,15,16,17,18,19,21,24,25,26,27] #list(np.arange(1,28, dtype=np.uint8))
gpu_id = 0
# ============================ #

run_france_all_images(splits_path='../data/splits/sherrie10k_planetImagery_splits_20x20_6x-downsampled.csv',
                      image_directory = '../data/planet/france/sherrie10k/monthly_mosaics_renamed_clipped_merged/2500px/6x_downsample/',
                      label_directory = '../data/planet/france/sherrie10k/extent_labels/2500px/6x_downsample/',
                      epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
                      n_classes=n_classes, model_type=model_type, month=month, image_suffix='.tif',
                      codes_to_keep=codes_to_keep, gpu_id=gpu_id, folder_suffix='_6x-downsampled',
                      boundary_kernel_size=(2,2))

Setting up a new session...


depth:= 0, nfilters: 16
depth:= 1, nfilters: 32
depth:= 2, nfilters: 64
depth:= 3, nfilters: 128
depth:= 4, nfilters: 256
depth:= 5, nfilters: 512
depth:= 6, nfilters: 256
depth:= 7, nfilters: 128
depth:= 8, nfilters: 64
depth:= 9, nfilters: 32
depth:= 10, nfilters: 16


Training epoch 2:   0%|          | 0/2562 [00:00<?, ?it/s]

Epoch 1:
    Train loss 0.213, accuracy 0.815, F1-score 0.654, MCC: 0.533, Dice: 0.654
    Val loss 0.214, accuracy 0.780, F1-score 0.651, MCC: 0.501, Dice: 0.651


Validation epoch 2: 100%|██████████| 585/585 [03:40<00:00,  2.65it/s]


...

Training epoch 17:   0%|          | 0/2562 [00:00<?, ?it/s]

Epoch 16:
    Train loss 0.179, accuracy 0.849, F1-score 0.720, MCC: 0.617, Dice: 0.720
    Val loss 0.179, accuracy 0.840, F1-score 0.720, MCC: 0.609, Dice: 0.720


Training epoch 18:   0%|          | 0/2562 [00:00<?, ?it/s]

Epoch 17:
    Train loss 0.179, accuracy 0.850, F1-score 0.721, MCC: 0.618, Dice: 0.721
    Val loss 0.180, accuracy 0.843, F1-score 0.723, MCC: 0.614, Dice: 0.723


Training epoch 18:   2%|▏         | 52/2562 [00:32<23:06,  1.81it/s] 

# Using Han and Burak's original sample locations

In [6]:
def run_france(epochs=100, lr=0.001, lr_decay=None, n_filters=16, batch_size=8,
               n_classes=1, model_type='resunet-d6', month='april', gpu_id=0,
               splits_path='../data/splits/hanAndBurak_planetImagery_splits.csv',
               n_train=1000):
    
    folder_name = model_type+'_'+month+'_ntrain-'+str(n_train)+'_nfilter-'+str(n_filters)+'_bs-'+str(batch_size)+'_lr-'+str(lr)
    if lr_decay:
        folder_name = folder_name + '_lrdecay-'+str(lr_decay)
    save_path = '../experiments/france/'+folder_name
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_model_name = os.path.join(save_path, "model.params")

    image_directory = '../data/planet/france/'+month
    label_directory = '../data/planet/france/labels/'

    env_name = 'france_'+folder_name
    vis = visdom.Visdom(port=8097, env=env_name)
    
    args = {}
    args['batch_size'] = batch_size
    args['gpu'] = gpu_id
    args['visdom'] = vis
    args['visdom_every'] = 10

    # Load train/val/test splits
    splits_df = pd.read_csv(splits_path)
    train_names = splits_df[splits_df['fold'] == 'train']['image_id'].values
    val_names = splits_df[splits_df['fold'] == 'val']['image_id'].values
    test_names = splits_df[splits_df['fold'] == 'test']['image_id'].values

    train_dataset = PlanetDataset(image_directory, label_directory, image_names=train_names)
    val_dataset = PlanetDataset(image_directory, label_directory, image_names=val_names)
    test_dataset = PlanetDataset(image_directory, label_directory, image_names=test_names)

    train_dataloader = gluon.data.DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = gluon.data.DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = gluon.data.DataLoader(test_dataset, batch_size=batch_size)

    # define model
    if model_type == 'resunet-d6':
        model = ResUNet_d6(_nfilters_init=n_filters, _NClasses=n_classes)
    elif model_type == 'resunet-d7':
        model = ResUNet_d7(_nfilters_init=n_filters, _NClasses=n_classes)
    model.initialize()
    model.hybridize()
    model.collect_params().reset_ctx(mx.gpu(args['gpu']))

    # define loss function
    tanimoto_dual = Tanimoto_wth_dual()
    if lr_decay:
        schedule = mx.lr_scheduler.FactorScheduler(step=1, factor=lr_decay)
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr, lr_scheduler=schedule)
    else:
        adam_optimizer = mx.optimizer.Adam(learning_rate=lr)
    trainer = gluon.Trainer(model.collect_params(), optimizer=adam_optimizer)

    # containers for metrics to log
    train_metrics = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                     'train_mcc': [], 'train_dice': []}
    val_metrics = {'val_loss': [], 'val_acc': [], 'val_f1': [], 
                   'val_mcc': [], 'val_dice': []}
    best_mcc = 0.0

    # training loop
    for epoch in range(1, epochs+1):

        # training set
        train_loss, train_accuracy, train_f1, train_mcc, train_dice = train_model(
            train_dataloader, model, tanimoto_dual, trainer, epoch, args)

        # training set metrics
        train_loss_avg = train_loss / len(train_dataset)
        train_metrics['train_loss'].append(train_loss_avg)
        train_metrics['train_acc'].append(train_accuracy.get()[1])
        train_metrics['train_f1'].append(train_f1.get()[1])
        train_metrics['train_mcc'].append(train_mcc.get()[1])
        train_metrics['train_dice'].append(train_dice.get()[1])

        # validation set
        val_loss, val_accuracy, val_f1, val_mcc, val_dice = evaluate_model(
            val_dataloader, model, tanimoto_dual, epoch, args)

        # validation set metrics
        val_loss_avg = val_loss / len(val_dataset)
        val_metrics['val_loss'].append(val_loss_avg)
        val_metrics['val_acc'].append(val_accuracy.get()[1])
        val_metrics['val_f1'].append(val_f1.get()[1])
        val_metrics['val_mcc'].append(val_mcc.get()[1])
        val_metrics['val_dice'].append(val_dice.get()[1])

        print("Epoch {}:".format(epoch))
        print("    Train loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            train_loss_avg, train_accuracy.get()[1], train_f1.get()[1], train_mcc.get()[1], train_dice.get()[1]))
        print("    Val loss {:0.3f}, accuracy {:0.3f}, F1-score {:0.3f}, MCC: {:0.3f}, Dice: {:0.3f}".format(
            val_loss_avg, val_accuracy.get()[1], val_f1.get()[1], val_mcc.get()[1], val_dice.get()[1]))

        # save model based on best MCC metric
        if val_mcc.get()[1] > best_mcc:
            model.save_parameters(save_model_name)
            best_mcc = val_mcc.get()[1]

        # save metrics
        metrics = pd.concat([pd.DataFrame(train_metrics), pd.DataFrame(val_metrics)], axis=1)
        metrics.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)

        # visdom
        vis.line(Y=np.stack([train_metrics['train_loss'], val_metrics['val_loss']], axis=1), 
                 X=np.arange(1, epoch+1), win="Loss", 
                 opts=dict(legend=['train loss', 'val loss'], markers=False, title="Losses",
                           xlabel="Epoch", ylabel="Loss")
                )
        vis.line(Y=np.stack([train_metrics['train_mcc'], val_metrics['val_mcc']], axis=1), 
                 X=np.arange(1, epoch+1), win="MCC", 
                 opts=dict(legend=['train MCC', 'val MCC'], markers=False, title="MCC",
                           xlabel="Epoch", ylabel="MCC")
                )


In [7]:
# ============================ #
# user-specified hyperparameters
# ============================ #
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 32
batch_size = 8
n_classes = 1
model_type = 'resunet-d7'
month = 'october'
gpu_id = 0
n_train = 500
splits_path = '../data/splits/hanAndBurak_planetImagery_splits_{}.csv'.format(n_train)
# ============================ #

run_france(epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
           n_classes=n_classes, model_type=model_type, month=month, gpu_id=gpu_id,
           splits_path=splits_path, n_train=n_train)

Setting up a new session...


depth:= 0, nfilters: 32
depth:= 1, nfilters: 64
depth:= 2, nfilters: 128
depth:= 3, nfilters: 256
depth:= 4, nfilters: 512
depth:= 5, nfilters: 1024
depth:= 6, nfilters: 2048
depth:= 7, nfilters: 1024
depth:= 8, nfilters: 512
depth:= 9, nfilters: 256
depth:= 10, nfilters: 128
depth:= 11, nfilters: 64
depth:= 12, nfilters: 32


Validation epoch 1: 100%|██████████| 51/51 [00:32<00:00,  1.57it/s]


Epoch 1:
    Train loss 0.284, accuracy 0.727, F1-score 0.749, MCC: 0.452, Dice: 0.749
    Val loss 0.247, accuracy 0.730, F1-score 0.716, MCC: 0.465, Dice: 0.716


Validation epoch 2: 100%|██████████| 51/51 [00:29<00:00,  1.72it/s]


...

Epoch 98:
    Train loss 0.148, accuracy 0.869, F1-score 0.879, MCC: 0.727, Dice: 0.879
    Val loss 0.188, accuracy 0.793, F1-score 0.809, MCC: 0.575, Dice: 0.809


Training epoch 100:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 99:
    Train loss 0.148, accuracy 0.868, F1-score 0.879, MCC: 0.727, Dice: 0.879
    Val loss 0.185, accuracy 0.800, F1-score 0.803, MCC: 0.591, Dice: 0.803


Validation epoch 100: 100%|██████████| 51/51 [00:27<00:00,  1.83it/s]

Epoch 100:
    Train loss 0.147, accuracy 0.867, F1-score 0.878, MCC: 0.724, Dice: 0.878
    Val loss 0.187, accuracy 0.801, F1-score 0.808, MCC: 0.591, Dice: 0.808


In [8]:
# ============================ #
# user-specified hyperparameters
# ============================ #
epochs = 100
lr = 0.001
lr_decay = None
n_filters = 32
batch_size = 8
n_classes = 1
model_type = 'resunet-d7'
month = 'october'
gpu_id = 0
n_train = 800
splits_path = '../data/splits/hanAndBurak_planetImagery_splits_{}.csv'.format(n_train)
# ============================ #

run_france(epochs=epochs, lr=lr, lr_decay=lr_decay, n_filters=n_filters, batch_size=batch_size,
           n_classes=n_classes, model_type=model_type, month=month, gpu_id=gpu_id,
           splits_path=splits_path, n_train=n_train)

Setting up a new session...


depth:= 0, nfilters: 32
depth:= 1, nfilters: 64
depth:= 2, nfilters: 128
depth:= 3, nfilters: 256
depth:= 4, nfilters: 512
depth:= 5, nfilters: 1024
depth:= 6, nfilters: 2048
depth:= 7, nfilters: 1024
depth:= 8, nfilters: 512
depth:= 9, nfilters: 256
depth:= 10, nfilters: 128
depth:= 11, nfilters: 64
depth:= 12, nfilters: 32


Validation epoch 1: 100%|██████████| 51/51 [00:28<00:00,  1.79it/s]


Epoch 1:
    Train loss 0.261, accuracy 0.740, F1-score 0.765, MCC: 0.477, Dice: 0.765
    Val loss 0.228, accuracy 0.757, F1-score 0.774, MCC: 0.502, Dice: 0.774


Validation epoch 2: 100%|██████████| 51/51 [00:28<00:00,  1.81it/s]


...

Epoch 98:
    Train loss 0.147, accuracy 0.867, F1-score 0.880, MCC: 0.723, Dice: 0.880
    Val loss 0.182, accuracy 0.811, F1-score 0.820, MCC: 0.612, Dice: 0.820


Training epoch 100:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 99:
    Train loss 0.147, accuracy 0.868, F1-score 0.881, MCC: 0.725, Dice: 0.881
    Val loss 0.182, accuracy 0.808, F1-score 0.825, MCC: 0.608, Dice: 0.825


Validation epoch 100: 100%|██████████| 51/51 [00:28<00:00,  1.80it/s]

Epoch 100:
    Train loss 0.147, accuracy 0.868, F1-score 0.881, MCC: 0.725, Dice: 0.881
    Val loss 0.183, accuracy 0.810, F1-score 0.818, MCC: 0.610, Dice: 0.818
